In [ ]:
!pip install keras
import keras
keras.__version__

# ShallowNet

In this exercise, we’ll implement the ShallowNet architecture. As the name suggests, the
ShallowNet architecture contains only a few layers – the entire network architecture can be
summarized as: INPUT => CONV => RELU => FC.
This simple network architecture will allow us to get our feet wet implementing Convolutional
Neural Networks using the Keras library. After implementing ShallowNet, We will apply it to the
CIFAR-10 dataset. 

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
#from pyimagesearch.nn.conv import ShallowNet
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

The lines above import our required Python packages. 

Let's have a look at our dataset:





In [ ]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0
print("[INFO] Finished loading CIFAR-10 data...")

Just like MNIST, CIFAR-10 is considered another standard benchmark dataset for image classification
in the computer vision and machine learning literature. CIFAR-10 consists of 60,000
32X32X3 (RGB) images resulting in a feature vector dimensionality of 3072.
As the name suggests, CIFAR-10 consists of 10 classes, including: airplanes, automobiles,
birds, cats, deer, dogs, frogs, horses, ships, and trucks.
While it’s quite easy to train a model that obtains > 97% classification accuracy on MNIST,
it’s substantially harder to obtain such a model for CIFAR-10 (and it’s bigger brother, CIFAR-100).

The challenge comes from the dramatic variance in how objects appear. For example, we can
no longer assume that an image containing a green pixel at a given (x;y)-coordinate is a frog. This
pixel could be part of the background of a forest that contains a deer. Or, the pixel could simply be
the color of a green truck.
These assumptions are a stark contrast to the MNIST dataset where the network can learn assumptions
regarding the spatial distribution of pixel intensities. For example, the spatial distribution
of foreground pixels of the number 1 is substantially different than a 0 or 5.
While being a small dataset, CIFAR-10 is still regularly used to benchmark new CNN architectures.
We load the training and testing data, then scale it into the range [0,1] .



Let's have a look at the train data:

In [ ]:
trainX.shape

In [ ]:
trainY.shape

Example of the 10 first train labels:

In [ ]:
trainY[0:10,:]

Example of the second image in the dataset (truck):

In [ ]:
%matplotlib inline
plt.figure
plt.imshow(trainX[1,:,:,:])

Let's have a look at the test data:

In [ ]:
testX.shape

In [ ]:
testY.shape

Example of the 10 first test labels:

In [ ]:
testY[0:10,:]

Our labels are then one-hot encoded from integers to vectors :

In [ ]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
trainY.shape

In [ ]:
trainY[0:10,:]

In [ ]:
testY.shape

In [ ]:
testY[0:10,:]

Initialize the label names for the CIFAR-10 dataset

In [ ]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
 "dog", "frog", "horse", "ship", "truck"]

Our workflow will be as follow: first we will present our neural network with the training data, `trainX` and `trainY`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `testX`, and we 
will verify if these predictions match the labels from `testY`.

Let's build our network:

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras import layers
from keras import models

# initialize the Shallownet model : 
#INPUT => 32 filters of 3X3 CONV => RELU => Flatten =>  FC with 10 neurons => softmax
 
#keras help: 
#https://keras.io/getting-started/sequential-model-guide/
#https://keras.io/layers/convolutional/  
#https://keras.io/layers/core/

model = Sequential()
inputShape = (32, 32, 3)


#Fill the lines below to complete the model... 



#show model summary:
model.summary()



The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of 'Conv2D' and 'Dense' ("fully-connected") layers. The last layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current image belongs to one of the 10 CIFAR-10 classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).



In [ ]:
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

In [ ]:
print("[INFO] training network...")
num_epochs = 5 # use less epochs in case of CPU installation
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	batch_size=32, epochs=num_epochs, verbose=1)
model.save('ShallowNetModel.h5')

Two quantities are being displayed during training: the "loss" of the network over the training/validation data, and the accuracy of the network over 
the training/validation data.

Now let's check that our model performs well on the validation set:

In [ ]:
##optional - load model from saved model
#from keras.models import load_model
#del model 
#model = load_model('ShallowNetModel.h5')

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, num_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, num_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num_epochs), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, num_epochs), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

After the training session is completed, 'play' with the following hyper-parameters 
(notice the accuracy, loss, training time):

1. Remove the Relu layer and see what happens
2. Learning rate- (0.1, 0.01, 0.001)
3. Batch Size - (1,32,64)
4. Number of epochs - (1,40,80)




Reference: 

Adrian Rosebrock, Deep Learning For Computer Vision With Python -Starter Bundle (2018), Chapter 12 